<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Asset Management

### The Basics (with `pandas`)

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

## Portfolio Theory

Topics of interest include:

* basics
* portfolio return
* portfolio risk
* possible risk-return combinations
* optimal portfolios
 * maximum Sharpe portfolio
 * minimum risk portfolio
 * minimum allocation portfolio
 * Expected Utility
 * Time-Additive Expected Utility

## More Realistic Model Economy

**_Two dates only (today, tomorrow), three traded assets and three future states with equal probability._**

## Basics

### Financial Assets

In [ ]:
import numpy as np
np.set_printoptions(suppress=True)

In [ ]:
M = np.array((
    (17, 10, 5),
    (20, 17, 2),
    (19, 0, 15)
)).T
M

In [ ]:
np.linalg.matrix_rank(M)

In [ ]:
M0 = np.ones(3) * 10
M0

### `pandas`

In [ ]:
import pandas as pd

In [ ]:
M = pd.DataFrame(M)

In [ ]:
M

In [ ]:
M.columns = [f'asset_{a}' for a in range(len(M.columns))]

In [ ]:
M.index = [f'state_{s}' for s in range(len(M))]

In [ ]:
M

### Probability

In [ ]:
P = np.array(3 * [1 / 3])
P

### Expectation

In [ ]:
np.dot(P, M)

### Expected Return

In [ ]:
R = M / M0 - 1
R

In [ ]:
R.mean(axis=0)

In [ ]:
R.mean()

### Volatility

In [ ]:
R.std(axis=0, ddof=0)

In [ ]:
R.std(ddof=0)

## Portfolio Return

In [ ]:
noa = M.shape[1]
noa

In [ ]:
def port_return(phi):
    return np.dot(R.mean(), phi)

In [ ]:
phi = np.ones(noa) / noa
phi

In [ ]:
port_return(phi)

## Portfolio Volatility

In [ ]:
R.cov()

In [ ]:
R.cov(ddof=0)

In [ ]:
np.diag(R.cov(ddof=0))  # variances

In [ ]:
np.diag(R.cov(ddof=0)) ** 0.5  # standard deviations (volatilities)

In [ ]:
def port_risk(phi):
    return np.dot(phi, np.dot(R.cov(ddof=0), phi)) ** 0.5

In [ ]:
port_risk(phi)

## Mean-Variance Portfolios

In [ ]:
p = np.random.random((1000, 3))
p = (p.T / p.sum(axis=1)).T
p.sum(axis=1)[:10]

In [ ]:
rr = [(port_risk(phi), port_return(phi)) for phi in p]

In [ ]:
rr = np.array(rr)
rr[:5]

In [ ]:
from pylab import plt
plt.style.use('seaborn-v0_8')

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(rr[:, 0], rr[:, 1], 'ro')
plt.xlabel('risk')
plt.ylabel('return');

## Optimal Portfolios

In [ ]:
from scipy.optimize import minimize

### Maximum Sharpe Ratio

In [ ]:
def sharpe(phi):
    return port_return(phi) / port_risk(phi)

In [ ]:
sharpe(phi)

In [ ]:
bnds = noa * [(0, 1)]
cons = {'type': 'eq', 'fun': lambda phi: phi.sum() - 1}

In [ ]:
opt = minimize(lambda phi: -sharpe(phi), phi, bounds=bnds, constraints=cons)

In [ ]:
opt

In [ ]:
phi_ = opt['x']

In [ ]:
np.dot(M, phi_)

In [ ]:
port_return(phi_)

In [ ]:
port_risk(phi_)

In [ ]:
sharpe(phi_)

### Minimum Risk Portfolio

In [ ]:
opt = minimize(port_risk, phi, bounds=bnds, constraints=cons)

In [ ]:
opt

In [ ]:
phi_ = opt['x']

In [ ]:
port_return(phi_)

In [ ]:
port_risk(phi_)

In [ ]:
sharpe(phi_)

In [ ]:
np.dot(M, phi_)

### Minimum Allocation Portfolio

In [ ]:
bnds = noa * [(0.15, 1)]

In [ ]:
opt = minimize(lambda phi: -sharpe(phi), phi, bounds=bnds, constraints=cons)

In [ ]:
opt

In [ ]:
phi_ = opt['x']

In [ ]:
port_return(phi_)

In [ ]:
port_risk(phi_)

In [ ]:
sharpe(phi_)

In [ ]:
type(M)

In [ ]:
np.dot(M, phi_)

### Expected Utility

In [ ]:
def u(x):
    return np.sqrt(x)

In [ ]:
def EU(phi):
    c = np.dot(M, phi)
    return np.dot(P, u(c))

In [ ]:
phi = np.ones(noa) / noa
phi

In [ ]:
EU(phi)

In [ ]:
bnds = noa * [(0, 1)]
bnds

In [ ]:
w = 10

In [ ]:
cons = {'type': 'eq', 'fun': lambda phi: np.dot(M0, phi) - w}

In [ ]:
opt = minimize(lambda phi: -EU(phi), phi, bounds=bnds, constraints=cons)

In [ ]:
opt

In [ ]:
phi_ = opt['x']

In [ ]:
port_return(phi_)

In [ ]:
port_risk(phi_)

In [ ]:
sharpe(phi_)

### Time-Additive Expected Utility

In [ ]:
M0 = np.array((1, 10, 10, 10))
M0

In [ ]:
kappa = 0.9

In [ ]:
def EU(phi):
    c = np.dot(M, phi[1:])
    return u(phi[0]) + kappa * np.dot(P, u(c))

In [ ]:
phi = np.random.random(4)
phi /= phi.sum()
phi

In [ ]:
bnds = [(0, None)] + 3 * [(0.0, None)]
bnds

In [ ]:
EU(phi)

In [ ]:
opt = minimize(lambda phi: -EU(phi), phi, bounds=bnds, constraints=cons)

In [ ]:
opt

In [ ]:
np.dot(M, opt['x'][1:])

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="30%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>